# nDCG (Normalized Discounted Cumulative Gain)  
The theoretical part include information from [Wikipedia page on nDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) with additional Python code to illustrate computations

- Ranging - sorting documents in order of their relevance for specific question. In our case, question comes from students and documents are recommended videos  
- Relevance - correspondence of recommended video to the students question. Further in this case, we will use Expert Evaluation from 0 to 10 as Relevance Metric

# nDCG (Normalized Discounted Cumulative Gain)  
nDCG - popular metric in Ranging Tasks, which includes order of elements in the output.  
We will introduce this metric step by step:

![](./pic/1.png)

2 assumptions are made in using DCG and its related measures:  
1. Highly relevant documents are more useful when appearing earlier in a search engine result list (have higher ranks)
2. Highly relevant documents are more useful than marginally relevant documents, which are in turn more useful than non-relevant documents.

- Cumulative Gain (CG) - is sum of the graded relevance values of all results in a search result list. it does not include Rank (position) of a result in the list.  
CG value is not affected by changes in the ordering of search results. That is problem of this metric

- Discounted Cumulative Gain - highly relevant documents appearing lower in a search result should be penalized as the graded relevance as the graded relevance is reduced logarithmically proportional to the position of the result.

- Normalized DCG - Search result list vary in length depending on the query. Comparing a search engine's performance from one query to the next cannot be consistently achieved using DCG alone, so the cumulative gain at each position for a chosen value oof p should be normalized across queries.  
This is done by sorting all relevant documents in the corpus by their relative relevance, producing the maximum possible DCG through position p, also called Ideal DCG through that position.

### Example:  
Presented with a list of documents in response to a search query, an experiment participant is asked to judge the relevance of each document to the query. Each document is to be judged on a scale of 0-3 with - meaning not relevant, 3 meaning highly relevant, and 1 and 2 meaning "somewhere in between".  
For the documents ordered by the ranking algorithm as D1, D2 .. D6  
the user provided the following relevance scores: 3, 2, 3, 0, 1, 2

That is: document 1 has relevance = 3 etc ... The CG (Cumulative Gain) of this search result:

In [82]:
# libraries:
import pandas as pd
import numpy as np

In [83]:
# example table:
documents = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6']
relevances = [3, 2, 3, 0, 1, 2]
orders = [1, 2, 3, 4, 5, 6]
df_relev = pd.DataFrame({
    'documents': documents,
    'relevance': relevances,
    'order': orders
})
df_relev

,documents,relevance,order
0,D1,3,1
1,D2,2,2
2,D3,3,3
3,D4,0,4
4,D5,1,5
5,D6,2,6


In [84]:
# CG:
CG_value = df_relev['relevance'].sum()
print(f'CG = {CG_value}')

CG = 11


we can see that changing the order of documents would not affect CG value.

DCG is used to emphasize highly relevant documents appearing early in the result list. Using logarithmic scale for reduction, the DCG for each result in order is:

In [85]:
df_relev['log2(i+1)'] = np.log2(df_relev['order'] + 1)
df_relev['dcg'] = df_relev['relevance'] / df_relev['log2(i+1)']
df_relev

,documents,relevance,order,log2(i+1),dcg
0,D1,3,1,1.000000,3.000000
1,D2,2,2,1.584963,1.261860
2,D3,3,3,2.000000,1.500000
3,D4,0,4,2.321928,0.000000
4,D5,1,5,2.584963,0.386853
5,D6,2,6,2.807355,0.712414


In [86]:
# by using function
def dcg_i(relevance, order):
    log_2_i_1 = np.log2(order + 1)
    rel_1_div_log_2 = round(relevance / log_2_i_1, 3)
    return rel_1_div_log_2

df_relev['DCG'] = df_relev.apply(lambda row: dcg_i(
    row['relevance'], 
    row['order']), axis = 1)

df_relev

,documents,relevance,order,log2(i+1),dcg,DCG
0,D1,3,1,1.000000,3.000000,3.000
1,D2,2,2,1.584963,1.261860,1.262
2,D3,3,3,2.000000,1.500000,1.500
3,D4,0,4,2.321928,0.000000,0.000
4,D5,1,5,2.584963,0.386853,0.387
5,D6,2,6,2.807355,0.712414,0.712


In [87]:
# DCG sum:
DCG_6 = df_relev['DCG'].sum()
DCG_6

6.861000000000001

now a switch D3 and D4 would result in a reduce DCG, because a less relevant document is placed higher in the ranking. Thank is a more relevant documnet is discounted more by being placed in a lower rank.

In [88]:
documents = ['D1', 'D2', 'D4', 'D3', 'D5', 'D6']  # D3 and D4 swapped
relevances = [3, 2, 0, 3, 1, 2]   # relevances updated to match new document order

df_relev_new = pd.DataFrame({
    'documents': documents,
    'relevance': relevances
})

df_relev_new['order'] = df_relev_new.index + 1
df_relev_new['log2(i+1)'] = np.log2(df_relev_new['order'] + 1)
df_relev_new['dcg'] = df_relev_new['relevance'] / df_relev_new['log2(i+1)']
dcg_new = df_relev_new['dcg'].sum()
print(f'DCG value new = {dcg_new}')

DCG value new = 6.653156362813681


The performance of this query to another is incomparable in this form since the other query may have more results, resulting in a larger overall DCG which may not necessarily be better. In order to compare, the DCG values must be normalized.  

To normalize DCG values, an ideal ordering for the given query is needed. For this example, that ordering would be **monotonically decreasing** sort of all known relevance judgments. In addition to the 6 from this experiment, suppose we also know there is a document D7 with relevance = 3 to the same query and document D8 with relevance = 2.  
The ideal ordering in that case would be:  

In [102]:
# example table:
documents = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8']
relevances = [3, 2, 3, 0, 1, 2, 3, 2]
orders = [1, 2, 3, 4, 5, 6, 7, 9]
df_relev_longer = pd.DataFrame({
    'documents': documents,
    'relevance': relevances,
    'order': orders
})

ideal_ordering = list(df_relev_longer.sort_values(by='relevance', ascending=False)['relevance'])
ideal_ordering

[3, 3, 3, 2, 2, 2, 1, 0]

- Ideal Ranking - is cut again to length 6 to match the depth of analysis of the ranking

In [104]:
df_cut = df_relev_longer[df_relev_longer['relevance'] >= 2]
df_cut = df_cut.sort_values(by='relevance', ascending=False)
ideal_ranking = list(df_cut['relevance'])
ideal_ranking

[3, 3, 3, 2, 2, 2]

The DCG of this ideal ordering (IDCG - ideal DCG) is computed to rank 6:

In [107]:
df_cut
df_cut['order'] = df_relev_new.index + 1
df_cut['log2(i+1)'] = np.log2(df_cut['order'] + 1)
df_cut['dcg'] = df_cut['relevance'] / df_cut['log2(i+1)']
dcg_ideal = df_cut['dcg'].sum()
print(f'DCG value new = {dcg_ideal}')

DCG value new = 8.740262365546284


and so the nDCG = $\frac{DCG6}{IDCG6} = \frac{6.861}{8.740} = 0.785$

In [112]:
nDCG = round(DCG_6 / dcg_ideal, 3)
print(f'nDCG = {nDCG}')

nDCG = 0.785
